In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('drive/My Drive/DataDays/second_round')

In [0]:
df = pd.read_csv('divar_posts_dataset.csv')

In [0]:
df['cat1_2'] = df.cat1.astype(str).str.cat(df.cat2.astype(str), sep='|')
df['cat1_2_3'] = df.cat1_2.astype(str).str.cat(df.cat3.astype(str), sep='|')

In [0]:
train,test = train_test_split(df,random_state = 100)

In [0]:
labels = {j:i for i,j in enumerate(train.cat1_2_3.unique())}
labels_cat1 = {j:i for i,j in enumerate(train.cat1.unique())}
labels_cat2 = {j:i for i,j in enumerate(train.cat2.unique())}
labels_cat3 = {j:i for i,j in enumerate(train.cat3.unique())}

In [0]:
X_train = train.apply(lambda i : i.title + ' ' + i.desc + ' '+i.city + ' '+i.created_at, axis=1)
Y_train = train.apply(lambda i : labels[i.cat1_2_3], axis=1)
Y_train_cat1 = train.apply(lambda i : labels_cat1[i.cat1], axis=1)
Y_train_cat2 = train.apply(lambda i : labels_cat2[i.cat2], axis=1)
Y_train_cat3 = train.apply(lambda i : labels_cat3[i.cat3], axis=1)


In [0]:
df.columns

Index(['Unnamed: 0', 'archive_by_user', 'brand', 'cat1', 'cat2', 'cat3',
       'city', 'created_at', 'desc', 'id', 'image_count', 'mileage',
       'platform', 'price', 'title', 'type', 'year', 'cat1_2', 'cat1_2_3'],
      dtype='object')

In [0]:
X_test = test.apply(lambda i : i.title + ' ' + i.desc + ' '+i.city + ' '+i.created_a, axis=1)
Y_test = test.apply(lambda i : labels[i.cat1_2_3], axis=1)
Y_test_cat1 = test.apply(lambda i : labels_cat1[i.cat1], axis=1)
Y_test_cat2 = test.apply(lambda i : labels_cat2[i.cat2], axis=1)
Y_test_cat3 = test.apply(lambda i : labels_cat3[i.cat3], axis=1)


In [0]:
output_test_cat1

array([0, 1, 3, ..., 5, 5, 1])

In [0]:
MAX_NB_WORDS = 30000

texts_train =X_train.astype(str)
texts_test = X_test.astype(str)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, char_level=False)
tokenizer.fit_on_texts(texts_train)
sequences = tokenizer.texts_to_sequences(texts_train)
sequences_test = tokenizer.texts_to_sequences(texts_test)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 438592 unique tokens.


In [0]:
index_to_word = dict((i, w) for w, i in tokenizer.word_index.items())

In [0]:
" ".join([index_to_word[i] for i in sequences[0]])

'پیش دستی و پلوخوری خارجی باسلام یه دست پیش دستی و یه دست پلوخوری که هرکدام مبلغش بیست و پنج هزارتومان میباشد با تخفیف باتشکر qom sunday 08am'

In [0]:
from keras.preprocessing.sequence import pad_sequences

In [0]:
MAX_SEQUENCE_LENGTH = 250

x_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', x_train.shape)
print('Shape of data test tensor:', x_test.shape)


Shape of data tensor: (710726, 250)
Shape of data test tensor: (236909, 250)


In [0]:


Y_train = to_categorical(np.asarray(Y_train))
Y_train_cat1 = to_categorical(np.asarray(Y_train_cat1))
Y_train_cat2 = to_categorical(np.asarray(Y_train_cat2))
Y_train_cat3 = to_categorical(np.asarray(Y_train_cat3))

Y_test = to_categorical(np.asarray(Y_test))
Y_test_cat1 = to_categorical(np.asarray(Y_test_cat1))
Y_test_cat2 = to_categorical(np.asarray(Y_test_cat2))
Y_test_cat3 = to_categorical(np.asarray(Y_test_cat3))

print('Shape of label tensor:', Y_train_cat1.shape)
print('Shape of label tensor:', Y_train_cat2.shape)
print('Shape of label tensor:', Y_train_cat3.shape)

print('Shape of label tensor:', Y_test_cat1.shape)
print('Shape of label tensor:', Y_test_cat2.shape)
print('Shape of label tensor:', Y_test_cat3.shape)




Shape of label tensor: (710726, 6)
Shape of label tensor: (710726, 28)
Shape of label tensor: (710726, 67)
Shape of label tensor: (236909, 6)
Shape of label tensor: (236909, 28)
Shape of label tensor: (236909, 67)


In [0]:
Y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [0]:
from keras.layers import Dense, Input, Flatten
from keras.layers import GlobalAveragePooling1D, Embedding , Reshape,Concatenate
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

EMBEDDING_DIM = 100
N_CLASSES_Cat1 = 6
N_CLASSES_Cat2 = 28
N_CLASSES_Cat3 = 67
filter_sizes = [2,3,4,5,6,7]
num_filters = 512
embedding_dim = 100
drop = 0.5

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedding_layer = Embedding(MAX_NB_WORDS, EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
embedded_sequences = embedding_layer(sequence_input)
reshape = Reshape((MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,1))(embedded_sequences)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_3 = Conv2D(num_filters, kernel_size=(filter_sizes[3], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_4 = Conv2D(num_filters, kernel_size=(filter_sizes[4], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_5 = Conv2D(num_filters, kernel_size=(filter_sizes[5], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)


concatenated_tensor = Concatenate(axis=1)([conv_0, conv_1, conv_2,conv_3,conv_4,conv_5])
flatten = Flatten()(concatenated_tensor)
x = Dropout(drop)(flatten)

predictions_cat1 = Dense(N_CLASSES_Cat1,name='cat1',activation='softmax')(x)
predictions_cat2 = Dense(N_CLASSES_Cat2,name='cat2',activation='softmax')(x)
predictions_cat3 = Dense(N_CLASSES_Cat3,name='cat3',activation='softmax')(x)
predictions = Dense(87,name='cat',activation='softmax')(x)



model = Model(inputs=sequence_input, outputs=[predictions_cat1,predictions_cat2,predictions_cat3,predictions])
loss_funcs = {
        "cat1": "categorical_crossentropy",
        "cat2": "categorical_crossentropy",
        "cat3": "categorical_crossentropy",
        "cat" : "categorical_crossentropy"
}
metrics = {
        "cat1": ['acc'],
        "cat2": ['acc'],
        "cat3": ['acc'],
        "cat" : ['acc']
}
y_trains = {
        "cat1": Y_train_cat1,
        "cat2": Y_train_cat2,
        "cat3": Y_train_cat3,
        "cat" : Y_train
}
y_valids = {
        "cat1": Y_test_cat1,
        "cat2": Y_test_cat2,
        "cat3": Y_test_cat3,
        "cat" : Y_test
}
model.compile(optimizer='adam',loss=loss_funcs, metrics=metrics)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 250)          0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 250, 100)     3000000     input_6[0][0]                    
__________________________________________________________________________________________________
reshape_3 (Reshape)             (None, 250, 100, 1)  0           embedding_6[0][0]                
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 249, 1, 512)  102912      reshape_3[0][0]                  
__________________________________________________________________________________________________
conv2d_10 

In [0]:
model.fit(x_train, y_trains, 
                        batch_size= 256,
                        epochs=10,
                        validation_data=(x_test,y_valids),
                        shuffle=True)

Train on 710726 samples, validate on 236909 samples
Epoch 1/10
  1536/710726 [..............................] - ETA: 51:28 - loss: 0.4346 - cat1_loss: 0.0748 - cat2_loss: 0.1017 - cat3_loss: 0.1339 - cat_loss: 0.1242 - cat1_acc: 0.9753 - cat2_acc: 0.9688 - cat3_acc: 0.9590 - cat_acc: 0.9622

KeyboardInterrupt: ignored

In [0]:
[output_test_cat1,output_test_cat2,output_test_cat3,output_test] = model.predict(x_test)


In [0]:
Y_test

14

In [0]:
from sklearn.metrics import classification_report

#output_test_cat1 = np.argmax(output_test_cat1,axis=1)
#output_test_cat2 = np.argmax(output_test_cat2,axis=1)
#output_test_cat3 = np.argmax(output_test_cat3,axis=1)
#output_test = np.argmax(output_test,axis=1)

#Y_test_cat1 = np.argmax(Y_test_cat1)
#Y_test_cat2 = np.argmax(Y_test_cat2)
#Y_test_cat3 = np.argmax(Y_test_cat3)
#Y_test = np.argmax(Y_test)

print(classification_report(Y_test_cat1, output_test_cat1))
print(classification_report(Y_test_cat2, output_test_cat2))
print(classification_report(Y_test_cat3, output_test_cat3))
print(classification_report(Y_test, output_test))

              precision    recall  f1-score   support

           0       0.94      0.97      0.96     72630
           1       0.97      0.97      0.97     51407
           2       0.95      0.94      0.95     34953
           3       0.96      0.97      0.97     41650
           4       0.95      0.93      0.94     20894
           5       0.85      0.75      0.80     15375

   micro avg       0.95      0.95      0.95    236909
   macro avg       0.94      0.92      0.93    236909
weighted avg       0.95      0.95      0.95    236909

              precision    recall  f1-score   support

           0       0.89      0.92      0.91     14761
           1       0.94      0.88      0.91      4976
           2       0.94      0.96      0.95     21795
           3       0.91      0.91      0.91      6527
           4       0.93      0.90      0.92     14016
           5       0.94      0.96      0.95     51014
           6       0.88      0.86      0.87       983
           7       0.97 

In [0]:
tst = pd.read_csv('dataset.csv')

In [0]:
X_tst = tst.apply(lambda i : str(i.title) + ' ' + str(i.desc), axis=1)


In [0]:
texts_tst = X_tst.astype(str)


In [0]:
sequences_tst = tokenizer.texts_to_sequences(texts_tst)


In [0]:
MAX_SEQUENCE_LENGTH = 250

x_tst = pad_sequences(sequences_tst, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data test tensor:', x_tst.shape)

Shape of data test tensor: (200000, 250)


In [0]:
[output_tst_cat1,output_tst_cat2,output_tst_cat3,output_tst] = model.predict(x_tst)


In [0]:
pred_cat1=np.argmax(output_tst_cat1,axis=1)
pred_cat2=np.argmax(output_tst_cat2,axis=1)
pred_cat3=np.argmax(output_tst_cat3,axis=1)
pred_cat=np.argmax(output_tst,axis=1)


In [0]:
tst['cat1']=pred_cat1
tst['cat2']=pred_cat2
tst['cat3']=pred_cat3
tst['cat1_2_3']=pred_cat


In [0]:
merged_cat1 = {}
for key, value in labels_cat1.items():
    merged_cat1[value] = key

merged_cat2 = {}
for key, value in labels_cat2.items():
    merged_cat2[value] = key

merged_cat3 = {}
for key, value in labels_cat3.items():
    merged_cat3[value] = key

merged = {}
for key, value in labels.items():
    merged[value] = key


In [0]:
pred_cat1.shape


(200000,)

In [0]:
tst['cat1'] = tst['cat1'].map(merged_cat1)
tst['cat2'] = tst['cat2'].map(merged_cat2)
tst['cat3'] = tst['cat3'].map(merged_cat3)
tst['cat1_2_3'] = tst['cat1_2_3'].map(merged)

In [0]:
tst

,Unnamed: 0,archive_by_user,city,created_at,desc,id,image_count,platform,price,title,cat1,cat2,cat3,cat1_2_3
0,0,False,Tehran,Sunday 08AM,این غذاخوری رو یک هفته هستش که خریدم ولی بخاطر...,34715297594354,2,mobile,550000,میز ناهارخوری ۴ نفره,for-the-home,furniture-and-home-decore,tables-and-chairs,for-the-home|furniture-and-home-decore|tables-...
1,1,True,Tehran,Tuesday 07AM,تبلت GALAXY TAB A\r\r\r\n١٠ اینچ ١٦ گیگ فول اچ...,22490819336293,2,mobile,800000,تبلت سامسونگ,electronic-devices,mobile-tablet,tablet,electronic-devices|mobile-tablet|tablet
2,2,True,Karaj,Friday 07PM,بدلیل جابجایی فروش فوری,38961075859535,1,mobile,200000,میز تلویزیون و نمای دکوری,for-the-home,furniture-and-home-decore,tv-and-stereo-furniture,for-the-home|furniture-and-home-decore|tv-and-...
3,3,False,Mashhad,Wednesday 02PM,رو دسته های قیچی کارشوده و خیلی خوش دست و تیز ...,66611618840254,2,mobile,90000,قیچی جگوار نو و سالم,for-the-home,utility,industrial,businesses|equipments-and-machinery|industrial
4,4,True,Tehran,Wednesday 05AM,111سالم بی رنگ فنی سالم تخفیف بیمه کامل بیمه ت...,57968462641639,2,mobile,16500000,111سالم نقد و اقساط,vehicles,cars,light,vehicles|cars|light
5,5,True,Mashhad,Friday 10AM,یک کرباتی سبز کاکلی نر پرشی قلدر قیمت ۱۰۰تومان...,42912634227113,2,mobile,-1,کرواتی سبز اس کاکلی,leisure-hobbies,animals,birds,leisure-hobbies|animals|birds
6,6,False,Ahvaz,Saturday 12PM,یک عدد اتو موی پریتک با صفحه ی سرامیکی کاملا س...,2732678124107,1,mobile,35000,یک عدد اتو موی پریتک,personal,health-beauty,NaN,personal|health-beauty|nan
7,7,True,Ahvaz,Monday 06PM,سلام کلش لول 78 دارم با قیمت 15 هزار تومان و ی...,46590712331339,1,mobile,15000,اکانت کلش حتما بخونید,electronic-devices,game-consoles-and-video-games,NaN,electronic-devices|game-consoles-and-video-gam...
8,8,True,Isfahan,Monday 12PM,۶لیتری\r\r\r\nبرنجی و سالم\r\r\r\nمهر دار\r\r\...,61497948119345,2,mobile,300000,سماورزغالی خاص,leisure-hobbies,furniture-and-home-decore,antiques-and-art,for-the-home|furniture-and-home-decore|antique...
9,9,False,Isfahan,Tuesday 08PM,پارس مشکی دوگانه بدون ضربه فقط یه کم رنگ پریدگ...,30714425893440,2,mobile,14000000,پارس 82مشکی دو گانه بدون ضربه,vehicles,cars,light,vehicles|cars|light


In [0]:
tst = tst.drop(columns=['Unnamed: 0', 'archive_by_user', 'city', 'created_at', 'desc', 'id',
       'image_count', 'platform', 'price', 'title'],axis=1)


In [0]:
tst.replace(['NaN'], '', inplace=True)

In [0]:
tst_not_seperate = tst.drop(columns=['cat1','cat2','cat3'])
tst_seperate = tst.drop(columns=['cat1_2_3'])

In [0]:
cat_tmp = tst_not_seperate['cat1_2_3'].str.split("|",n=1,expand=True)
tst_not_seperate['cat1'] = cat_tmp[0]
tst_not_seperate['cat2_3']=cat_tmp[1] 
cat2_tmp = tst_not_seperate['cat2_3'].str.split("|",n=1,expand=True)
tst_not_seperate['cat2'] = cat2_tmp[0]
tst_not_seperate['cat3']=cat2_tmp[1] 

In [0]:
tst_not_seperate=tst_not_seperate.drop(columns=['cat1_2_3','cat2_3'])

In [0]:
tst_seperate

NameError: ignored

In [0]:
ne_stacked = (tst != df2).stack()

In [0]:
tst.to_csv("tst_deep_cat3_90.csv")

In [0]:
.sa